## Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Reshape
from tensorboard.plugins.hparams import api as hp
import pandas as pd
import pandas.util
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Hyperparameters

In [ ]:
IMG_SIZE = (105,75)    # width and height of all images (resize, if required)
BATCH_SIZE = 32  # for training and prediction
EPOCHS = 30 #number of epochs for training

HP_DROPOUT_CONV = hp.HParam('dropout_conv', hp.Discrete([.25,.3,.35,.4])) 
HP_DROPOUT_DENSE = hp.HParam('dropout_dense', hp.Discrete([.25,.3,.35,.4]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'

# get test, train, validation datasets

In [ ]:
test_df = pd.read_pickle("classification_test.pkl")
train_df = pd.read_pickle("classification_train.pkl")
val_df = pd.read_pickle("classification_val.pkl")

# Map a filename to an one-hot encode label

In [ ]:
def path_to_array(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_png(img, channels = 3)
    # now img is 3 dim array of numbers in {0,..., 255}
    img = tf.cast(img, dtype = tf.float32) / 255. # scale to floating point number in [0,1] EVTL IN [-1,1] skalieren??????
 
    # one-hot encode the label, e.g. 3 becomes [0,0,0,1,0,0,0,0,0,0,0,0]
    label = tf.one_hot(label, depth = 6) #6 classes
    return img, label

# Make a tf dataset of images from a pd data frame of file paths

In [ ]:
def make_dataset(df):
    # first, make dataset with just the relevant: path and class
    ds_path = tf.data.Dataset.from_tensor_slices((df['image_path'], df['class']))

    # convert to data set with actual images
    ds = ds_path.map(path_to_array)
    ds = ds.batch(BATCH_SIZE)
    return ds

test_ds  = make_dataset(test_df)
val_ds   = make_dataset(val_df)
train_ds = make_dataset(train_df)
train_ds = train_ds.repeat().prefetch(tf.data.experimental.AUTOTUNE) # infinitely repeat

# Determine the architecture of the model

In [ ]:
def create_model(hparams):
    
    kernel_size = (3, 3)
    pool_size   = (2, 2)
    first_filters  = 32
    second_filters = 64
    third_filters  = 128
    dropout_conv  = hparams[HP_DROPOUT_CONV]
    dropout_dense = hparams[HP_DROPOUT_DENSE]

    model = tf.keras.models.Sequential() # sequential stack of layers

    model.add( BatchNormalization(input_shape = (IMG_SIZE[1],IMG_SIZE[0], 3)))
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) # convolutional layer + activation layer
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) 
    model.add( Conv2D (first_filters, kernel_size, activation = 'relu')) 
    model.add( MaxPooling2D (pool_size = pool_size)) #Pooling layer
    model.add( Dropout (dropout_conv)) #Dropout layer

    model.add( Conv2D (second_filters, kernel_size, activation ='relu')) 
    model.add( Conv2D (second_filters, kernel_size, activation ='relu')) 
    model.add( Conv2D (second_filters, kernel_size, activation ='relu'))
    model.add( MaxPooling2D (pool_size = pool_size))
    model.add( Dropout (dropout_conv))

    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( Conv2D (third_filters, kernel_size, activation ='relu'))
    model.add( MaxPooling2D (pool_size = pool_size))
    model.add( Dropout (dropout_conv))

    model.add( Flatten())
    model.add( Dense (256, activation = "relu", kernel_regularizer = tf.keras.regularizers.l2(0.001))) #dense layer + activation layer
    model.add( Dropout (dropout_dense)) #Dropout layer
    model.add( Dense(6, activation = 'softmax') ) # activation layer

    return model

# Train the model via hyperparameter tuning

In [ ]:
num_train = len(train_df)
session_num = 0


for dr_conv in HP_DROPOUT_CONV.domain.values:
    for dr_dense in HP_DROPOUT_DENSE.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:

            hparams = {
                HP_DROPOUT_CONV: dr_conv,
                HP_DROPOUT_DENSE: dr_dense,
                HP_OPTIMIZER: optimizer,
            }
            run_name = f"run-{session_num}"
            run_dir = 'logs/hparam_tuning/' + run_name
            print(f'--- Starting trial: {run_name}')
            print({h.name: hparams[h] for h in hparams})


            with tf.summary.create_file_writer(run_dir).as_default():
                hp.hparams(hparams)  # record the values used in this trial
                model = create_model(hparams)
                # Function to decrease learning rate by 'factor'
                # when there has been no significant improvement in the last 'patience' epochs.
                reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', factor = 0.75, patience = 4, verbose = 1)
                         
                # define the loss, optimization algorithm and prepare the model for gradient computation 
                model.compile(optimizer = hparams[HP_OPTIMIZER],
                              loss = 'categorical_crossentropy', metrics = [METRIC_ACCURACY]) 

                # Callbacks: What should be done during (long) training?
                modelfname = f"model_checkpoints/classification_{dr_conv}_{dr_dense}_{optimizer}.h5"
                # Function to store model to file, if validation loss has a new record
                # Check always after having seen at least another save_freq examples.
                checkpoint = tf.keras.callbacks.ModelCheckpoint(
                    modelfname, monitor = 'val_loss', mode = 'min', 
                    save_best_only = True, verbose = 1)

                history = model.fit_generator(
                    train_ds, epochs = EPOCHS, 
                    steps_per_epoch = num_train / BATCH_SIZE, #would use each example once on average
                    validation_data = val_ds, verbose = 1,
                    callbacks = [checkpoint,reduce_lr]
                )

                # Das "beste" Model (kleinster val_loss) fuer die eval. benutzen
                model.load_weights(modelfname)
                _, accuracy = model.evaluate(test_ds)        
                tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

                session_num += 1
